# Использование адаптивного порога для алгоритма KMS

In [ ]:
```bash
for src in ./bin0427/*.bin; do
  for i in {1..10}; do
    python3 KMS_adaptive_threshold.py src:$bin dataset:10 seed:1337 repeats:2 &
  done
  wait
done
```

---

## Черновики:

In [1]:
import glob, json, os, warnings, random, math
import numpy as np
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.piecewise import SymbolicAggregateApproximation

root = './bin0427/' # HGDP DB

In [2]:
for src in sorted(glob.glob(f'{root}*.bin')):
    print(src)

./bin0427/HGDP_CNV_gain_L_filterd.bin
./bin0427/HGDP_CNV_gain_R_filterd.bin
./bin0427/HGDP_CNV_loss_L_filterd.bin
./bin0427/HGDP_CNV_loss_R_filterd.bin
./bin0427/HGDP_DEL_L_filterd.bin
./bin0427/HGDP_DEL_R_filterd.bin
./bin0427/HGDP_DUP_L_filterd.bin
./bin0427/HGDP_DUP_R_filterd.bin
./bin0427/HGDP_INS_C_filterd.bin
./bin0427/HGDP_INV_L_filterd.bin
./bin0427/HGDP_INV_R_filterd.bin


In [3]:
seed = 1337

def read_coverage(f, num = 0):
    fh = open(f, "rb")
    fh.seek(num * 1024)
    bin = fh.read(1024)
    return [(bin[i] * 256 + bin[i + 1]) for i in range(0, len(bin), 2)]


def read_random(f, count=100, seed=0):
    random.seed(seed)
    index_range = range(int(os.path.getsize(f)/1024))
    return [read_coverage(f, i) for i in random.sample(index_range, min(count, len(index_range) - 1))]


def basis(size, func):
    R = np.arange(0, 2 * math.pi, 2 * math.pi / size)
    return np.array([func(v) for v in R])


def results_save(subs, results):
    mtx = []
    for result in results[0:12]:
        mt_one = []
        for i in result:
            mt_one.append([subs[i].offset, [int(v[0]) for v in subs[i].src]])
        mtx.append(mt_one)
    return mtx

In [4]:
options = {
    'src': './bin0427/HGDP_INS_C_filterd.bin', 
    'dataset': 1000, 
    'sax': 64,
    'alphabet': 32,
    'window': 32
}

window_step = int(options['window']/8)

sax = SymbolicAggregateApproximation(
    n_segments=options['sax'],
    alphabet_size_avg=options['alphabet'])

sax_motif = SymbolicAggregateApproximation(
    n_segments=options['window'],
    alphabet_size_avg=options['alphabet'])

data = np.array(read_random(options['src'], options['dataset'], seed))
cls = TimeSeriesScalerMeanVariance().fit_transform(data)
trsf = sax.fit_transform(cls)
        
subs = []
for k, signal in enumerate(trsf):
    for i in range(0, len(signal) - options['window'] + 1, window_step):
        obj = {'raw': signal[i:(i + options['window'])], 'offset': i, 'src': trsf[k]}
        subs.append(type('new_dict', (object,), obj))

In [5]:
CACHE = {}

def saxdist(idx, idy):
    d_key = f'{min(idx,idy)}:{max(idx,idy)}'
    if d_key not in CACHE:
        CACHE[d_key] = sax.distance_sax(subs[idx].raw, subs[idy].raw)
    return CACHE[d_key]

In [6]:
INF = 999999

def clusters_dist(C1, C2):
    sm = INF
    for ixd in C1:
        for ixy in C2:
            sm = min(sm, saxdist(idx, idy))
    return sm

def diam(C):
    mx = 0
    for a in range(0, len(C) - 1):
        for b in range(a, len(C)):
            mx = max(mx, saxdist(C[a], C[b]))
    return mx


def dunn(results):
    if len(results) < 2:
        return -1
    total_items = sum([len(C) for C in results])
    D = np.mean([diam(C) for C in results])
    if D == 0: return [INF, total_items, len(results[0])]
    min_dist = INF
    for i in range(0, len(results) - 1):
        for j in range(i, len(results)):
            min_dist = min(min_dist, clusters_dist(results[i], results[j]))
    return [min_dist/D, total_items, len(results[0])]

In [7]:
cnt = len(subs)
dms = subs[0].raw.shape[0]

ref_A, ref_B = sax_motif.fit_transform([basis(dms, math.sin), basis(dms, math.cos)])
d2ref = np.array([[i, sax.distance_sax(obj.raw, ref_A), sax.distance_sax(obj.raw, ref_B)] for i, obj in enumerate(subs)]).reshape(cnt, 3)
argss = d2ref[:, 1].argsort()

In [8]:
from IPython.display import clear_output

In [18]:
np.arange(0,8,0.2)
[{'m': make_matrix(subs, result, sax_motif), 'items': len(result)} for result in results[0:5]]

array([0. , 0.2, 0.4, 0.6, 0.8, 1. , 1.2, 1.4, 1.6, 1.8, 2. , 2.2, 2.4,
       2.6, 2.8, 3. , 3.2, 3.4, 3.6, 3.8, 4. , 4.2, 4.4, 4.6, 4.8, 5. ,
       5.2, 5.4, 5.6, 5.8, 6. , 6.2, 6.4, 6.6, 6.8, 7. , 7.2, 7.4, 7.6,
       7.8])

In [19]:
def make_matrix(subs, result, sax):
    Z = np.zeros((sax.n_segments, sax.alphabet_size_avg))
    for i in result:
        for k, v in enumerate(subs[i].raw):
            Z[k][v[0]] += 1
    return [list(i) for i in Z/len(result)]


In [21]:
threshold = 5.0
min_index_dist = 4
dunn_list = []


while threshold < 8:
    threshold += 0.2
    clear_output(wait=True)
    print(threshold)
    links = np.array([-1 for i in range(cnt)])
    for i, idx in enumerate(argss[:-1]):
        for idy in argss[i+1:]:
            A, B = (d2ref[idx], d2ref[idy])
            if abs(A[0] - B[0]) < min_index_dist: continue
            if abs(A[1] - B[1]) > 2 * threshold: break
            if abs(A[2] - B[2]) > 2 * threshold: break
            dst = saxdist(idx, idy)
            if dst > threshold: continue
            
            parent = max(links[idx], links[idy])
            if parent == -1: 
                parent = max(idx, idy)
            for cng in [idx, idy]:
                if links[cng] == -1:
                    links[cng] = parent
                if links[cng] != parent:
                    links[links == links[cng]] = parent
    motif = {}
    for idx in links[links != -1]:
        if idx not in motif: motif[idx] = 0
        motif[idx] += 1
    
    siz = np.array([[motif[v], v] for v in motif])
    if len(siz) == 0:
        print('PASS')
        continue

    mtf = siz[(-siz[:,0]).argsort()]
    results = [np.where(links == idx)[0] for cnt, idx in mtf]
    dunn_list.append(dunn(results))
    test = [{'m': make_matrix(subs, result, sax_motif), 'items': len(result)} for result in results[0:5]]
    print(test)
    break
    # objx = results_save(subs, results[0:12])            

5.2
[{'m': [[0.02608695652173913, 0.05217391304347826, 0.11304347826086956, 0.12173913043478261, 0.16521739130434782, 0.1391304347826087, 0.12173913043478261, 0.08695652173913043, 0.06086956521739131, 0.043478260869565216, 0.017391304347826087, 0.017391304347826087, 0.008695652173913044, 0.008695652173913044, 0.008695652173913044, 0.008695652173913044, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.034782608695652174, 0.06956521739130435, 0.2, 0.17391304347826086, 0.1565217391304348, 0.0782608695652174, 0.0782608695652174, 0.08695652173913043, 0.06956521739130435, 0.02608695652173913, 0.0, 0.017391304347826087, 0.0, 0.008695652173913044, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0782608695652174, 0.12173913043478261, 0.21739130434782608, 0.1565217391304348, 0.12173913043478261, 0.0782608695652174, 0.06086956521739131, 0.06956521739130435, 0.06086956521739131, 0.0, 0.017391304347826087, 0.0, 0.0, 0.

In [ ]:



def find_KMS_v4(subs, seed, **kwargs):
    if not sax or not sax_motif: 
        return False

    random.seed(seed)
    cnt = len(subs)
    dms = subs[0].raw.shape[0]

    #ref_A = subs[random.choice(range(0, cnt))].raw
    #ref_B = subs[random.choice(range(0, cnt))].raw
    
    ref_A, ref_B = sax_motif.fit_transform([basis(dms, math.sin), basis(dms, math.cos)])
    # ref_A, ref_B = sax_motif.fit_transform([basis(dms, lambda f: -math.sin(f)), basis(dms, lambda f: -math.cos(f))])
    
    d2ref = np.array([[i, sax.distance_sax(obj.raw, ref_A), sax.distance_sax(obj.raw, ref_B)] for i, obj in enumerate(subs)]).reshape(cnt, 3)

    argss = d2ref[:, 1].argsort()
    threshold = 3
    min_index_dist = 4
    links = np.array([-1 for i in range(cnt)])
    default = np.array([[0] for i in range(options['window'])])

    for i, idx in enumerate(argss[:-1]):
        for idy in argss[i+1:]:
            A, B = (d2ref[idx], d2ref[idy])
            if abs(A[0] - B[0]) < min_index_dist: continue
            if abs(A[1] - B[1]) > 2 * threshold: break
            if abs(A[2] - B[2]) > 2 * threshold: break
            
            trv = sax.distance_sax(subs[idx].raw - np.min(subs[idx].raw), default)
            if trv < 2 * threshold: continue

            dst = sax.distance_sax(subs[idx].raw, subs[idy].raw)
            if dst > threshold: continue

            parent = max(links[idx], links[idy])
            if parent == -1: 
                parent = max(idx, idy)
            for cng in [idx, idy]:
                if links[cng] == -1:
                    links[cng] = parent
                if links[cng] != parent:
                    links[links == links[cng]] = parent
    motif = {}
    for idx in links[links != -1]:
        if idx not in motif: motif[idx] = 0
        motif[idx] += 1
    
    siz = np.array([[motif[v], v] for v in motif])
    if len(siz) == 0: return []

    mtf = siz[(-siz[:,0]).argsort()]
    results = [np.where(links == idx)[0] for cnt, idx in mtf]
    
    return results_save(subs, results[0:12])
    #return [{'m': make_matrix(subs, result, sax_motif), 'items': len(result)} for result in results[0:5]]
